In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import torch
from chronos import BaseChronosPipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

In [ ]:
# Parameters and Settings

# Parameters for data split
WINDOW = 21    # rolling window size to use as predictors
DATE_COL = 'Date'
ID_COL = 'PERMNO'
TARGET_COL = 'excess_return'

# File path for the cleaned and filtered data file
current_directory = os.getcwd()
clean_filtered_data_path = os.path.join(current_directory, 'Data', 'clean_filtered_data.csv')

# File path to save prediction results
results_path = os.path.join(current_directory, 'Results', f'chronos_models_results{WINDOW:.0f}.csv')

# Estimation (in sample) period dates
in_sample_start_date = pd.to_datetime("2000-01-01")
in_sample_end_date = pd.to_datetime("2015-12-31")

# Out-of-sample period dates
out_sample_start_date = pd.to_datetime("2016-01-01")
out_sample_end_date = pd.to_datetime("2024-12-31")

# Use GPU if available, else default to using CPU for Bolt models
device_map = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Step 1: Load and Process Data

In [ ]:
# Load the cleaned and filtered data files for in sample and out of sample periods into a pandas DataFrames
df = pd.read_csv(clean_filtered_data_path)

# Ensure the date columns are in datetime format
df[DATE_COL] = pd.to_datetime(df[DATE_COL])

df = df[[ID_COL, DATE_COL, TARGET_COL]].dropna()
df = df.sort_values([ID_COL, DATE_COL]).reset_index(drop=True)

df.info()

In [ ]:
# Check number of unique stocks
stocks_permno = df["PERMNO"].unique().tolist()
print(f"Number of unique stocks: {len(stocks_permno)}")

In [ ]:
# Spit data into estimation (in-sample) and out-of-sample data
df_train = df[(df[DATE_COL] >= in_sample_start_date) & (df[DATE_COL] <= in_sample_end_date)].copy().reset_index(drop=True)
out_sample_start_date = df_train[DATE_COL].tail(WINDOW).iloc[0]
df_test = df[(df[DATE_COL] >= (out_sample_start_date)) & (df[DATE_COL] <= out_sample_end_date)].copy().reset_index(drop=True)

print(df_train.info())
print(df_test.info())

In [ ]:
# Create rolling window for predictors for Bolt models
contexts = []
targets = []
records = []

for id, grp in df_test.groupby(ID_COL):
    values = grp[TARGET_COL].values
    dates  = grp[DATE_COL].values
    for i in range(len(values) - WINDOW):
        contexts.append(torch.tensor(values[i:i+WINDOW], dtype=torch.float32, device=device))
        targets.append(values[i+WINDOW])
        records.append({
            ID_COL: id,
            TARGET_COL: values[i+WINDOW],
            DATE_COL: dates[i+WINDOW]
        })

In [ ]:
y_test = pd.Series(targets)

results = pd.DataFrame(records)

### Step 2: Zero-Shot Forecasting with Chronos-Bolt Models

In [ ]:
# Creating a Function to Calculate Predictive-R2 Used in the Finance Literature
def r2(y_true, y_pred):
    return 1-(((y_true-y_pred)**2).sum()/(y_true**2).sum())

In [ ]:
# Zero Shot Chronos-Bolt-Tiny
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-bolt-tiny',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx.to(device)], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.cpu().squeeze().item())
y_chr_bolt_tiny = pd.Series(preds)

results['y_chr_bolt_tiny'] = y_chr_bolt_tiny

In [ ]:
# Zero Shot Chronos-Bolt-Mini
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-bolt-mini',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx.to(device)], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.cpu().squeeze().item())
y_chr_bolt_mini = pd.Series(preds)

results['y_chr_bolt_mini'] = y_chr_bolt_mini

In [ ]:
# Zero Shot Chronos-Bolt-Small
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-bolt-small',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx.to(device)], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.cpu().squeeze().item())
y_chr_bolt_small = pd.Series(preds)

results['y_chr_bolt_small'] = y_chr_bolt_small

In [ ]:
# Zero Shot Chronos-Bolt-Base
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-bolt-base',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx.to(device)], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.cpu().squeeze().item())
y_chr_bolt_base = pd.Series(preds)

results['y_chr_bolt_base'] = y_chr_bolt_base

### Step 3: Zero-Shot Forecasting with Chronos-T5 Models

In [ ]:
# Switching to using CPU for T5 models (system has lower GPU memory than required to load model)
device_map = "cpu"
device = torch.device("cpu")

In [ ]:
# Create rolling window for predictors for T5 models
contexts = []
targets = []

for id, grp in df_test.groupby(ID_COL):
    values = grp[TARGET_COL].values
    dates  = grp[DATE_COL].values
    for i in range(len(values) - WINDOW):
        contexts.append(torch.tensor(values[i:i+WINDOW], dtype=torch.float32, device=device))
        targets.append(values[i+WINDOW])

In [ ]:
# Zero Shot Chronos-T5-Tiny
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-t5-tiny',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.squeeze().item())
y_chr_t5_tiny = pd.Series(preds)

results['y_chr_t5_tiny'] = y_chr_t5_tiny

In [ ]:
# Zero Shot Chronos-T5-Mini
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-t5-mini',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.squeeze().item())
y_chr_t5_mini = pd.Series(preds)

results['y_chr_t5_mini'] = y_chr_t5_mini

In [ ]:
# Zero Shot Chronos-T5-Small
pipeline = BaseChronosPipeline.from_pretrained(
    'amazon/chronos-t5-small',
    device_map=device_map,
    torch_dtype=torch.float32
)

preds = []
for ctx in contexts:
    quantiles, mean = pipeline.predict_quantiles(context=[ctx], prediction_length=1, quantile_levels=[0.5])
    preds.append(mean.squeeze().item())
y_chr_t5_small = pd.Series(preds)

results['y_chr_t5_small'] = y_chr_t5_small

### Step 4: Evaluate Statistical Performance of Models

In [ ]:
# Evaluate models

# Chronos-Bolt-Tiny
r2_chr_bolt_tiny  = r2(y_test, y_chr_bolt_tiny)
mse_chr_bolt_tiny = mean_squared_error(y_test, y_chr_bolt_tiny)
mae_chr_bolt_tiny = mean_absolute_error(y_test, y_chr_bolt_tiny)
da_chr_bolt_tiny = (np.sign(y_test) == np.sign(y_chr_bolt_tiny)).mean()

# Chronos-Bolt-Mini
r2_chr_bolt_mini  = r2(y_test, y_chr_bolt_mini)
mse_chr_bolt_mini = mean_squared_error(y_test, y_chr_bolt_mini)
mae_chr_bolt_mini = mean_absolute_error(y_test, y_chr_bolt_mini)
da_chr_bolt_mini = (np.sign(y_test) == np.sign(y_chr_bolt_mini)).mean()

# Chronos-Bolt-Small
r2_chr_bolt_small  = r2(y_test, y_chr_bolt_small)
mse_chr_bolt_small = mean_squared_error(y_test, y_chr_bolt_small)
mae_chr_bolt_small = mean_absolute_error(y_test, y_chr_bolt_small)
da_chr_bolt_small = (np.sign(y_test) == np.sign(y_chr_bolt_small)).mean()

# Chronos-Bolt-Base
r2_chr_bolt_base  = r2(y_test, y_chr_bolt_base)
mse_chr_bolt_base = mean_squared_error(y_test, y_chr_bolt_base)
mae_chr_bolt_base = mean_absolute_error(y_test, y_chr_bolt_base)
da_chr_bolt_base = (np.sign(y_test) == np.sign(y_chr_bolt_base)).mean()

# Chronos-T5-Tiny
r2_chr_t5_tiny  = r2(y_test, y_chr_t5_tiny)
mse_chr_t5_tiny = mean_squared_error(y_test, y_chr_t5_tiny)
mae_chr_t5_tiny = mean_absolute_error(y_test, y_chr_t5_tiny)
da_chr_t5_tiny = (np.sign(y_test) == np.sign(y_chr_t5_tiny)).mean()

# Chronos-T5-Mini
r2_chr_t5_mini  = r2(y_test, y_chr_t5_mini)
mse_chr_t5_mini = mean_squared_error(y_test, y_chr_t5_mini)
mae_chr_t5_mini = mean_absolute_error(y_test, y_chr_t5_mini)
da_chr_t5_mini = (np.sign(y_test) == np.sign(y_chr_t5_mini)).mean()

# Chronos-T5-Small
r2_chr_t5_small  = r2(y_test, y_chr_t5_small)
mse_chr_t5_small = mean_squared_error(y_test, y_chr_t5_small)
mae_chr_t5_small = mean_absolute_error(y_test, y_chr_t5_small)
da_chr_t5_small = (np.sign(y_test) == np.sign(y_chr_t5_small)).mean()

In [ ]:
# Collating Results

results_matrix = [{
        "Model": "Chronos-Bolt-Tiny",
        "R-squared": r2_chr_bolt_tiny,
        "MSE": mse_chr_bolt_tiny,
        "MAE": mae_chr_bolt_tiny,
        "Direction Accuracy": da_chr_bolt_tiny
    },
    {
        "Model": "Chronos-Bolt-Mini",
        "R-squared": r2_chr_bolt_mini,
        "MSE": mse_chr_bolt_mini,
        "MAE": mae_chr_bolt_mini,
        "Direction Accuracy": da_chr_bolt_mini
    },
    {
        "Model": "Chronos-Bolt-Small",
        "R-squared": r2_chr_bolt_small,
        "MSE": mse_chr_bolt_small,
        "MAE": mae_chr_bolt_small,
        "Direction Accuracy": da_chr_bolt_small
    },
    {
        "Model": "Chronos-Bolt-Base",
        "R-squared": r2_chr_bolt_base,
        "MSE": mse_chr_bolt_base,
        "MAE": mae_chr_bolt_base,
        "Direction Accuracy": da_chr_bolt_base
    },
    {
        "Model": "Chronos-T5-Tiny",
        "R-squared": r2_chr_t5_tiny,
        "MSE": mse_chr_t5_tiny,
        "MAE": mae_chr_t5_tiny,
        "Direction Accuracy": da_chr_t5_tiny
    },
    {
        "Model": "Chronos-T5-Mini",
        "R-squared": r2_chr_t5_mini,
        "MSE": mse_chr_t5_mini,
        "MAE": mae_chr_t5_mini,
        "Direction Accuracy": da_chr_t5_mini
    },
    {
        "Model": "Chronos-T5-Small",
        "R-squared": r2_chr_t5_small,
        "MSE": mse_chr_t5_small,
        "MAE": mae_chr_t5_small,
        "Direction Accuracy": da_chr_t5_small
    }]

results_matrix_df = pd.DataFrame(results_matrix)
results_matrix_df

##### Save Results

In [ ]:
# Save Prediction Results
results.to_csv(results_path, index=False)